<a href="https://colab.research.google.com/github/Wally0924/Embedded-Systems/blob/main/homework/transportation/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## dataset

In [2]:
# TODO upload your dataset or use roboflow download code
!curl -L https://app.roboflow.com/ds/1vs3ErCF56?key=8EuGgsiV0M > roboflow.zip
!unzip ./roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   894  100   894    0     0    700      0  0:00:01  0:00:01 --:--:--   700
100 10.0M  100 10.0M    0     0  3182k      0  0:00:03  0:00:03 --:--:-- 6377k
Archive:  ./roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/youtube-0_jpg.rf.cefdef78147575c7dcdee4eaadf6978e.jpg  
 extracting: test/images/youtube-102_jpg.rf.360b3b2577d6d98dd4af8917bc431296.jpg  
 extracting: test/images/youtube-105_jpg.rf.ec6867fa02ded89a6f3d04087d6a05c7.jpg  
 extracting: test/images/youtube-120_jpg.rf.2452017d460cc54eae29f47dca6132a3.jpg  
 extracting: test/images/youtube-127_jpg.rf.071414e4c5d73e12c686d20e1281e6f9.jpg  
 extracting: test/images/youtube-136_jpg.rf.aeb6348170c695d9661ddfb5dd550c95.jpg  
 ext

### train model

In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 10.5 MB/s eta 0:00:00


In [4]:
# TODO train the model with your dataset
# recommend at least train 50 epochs
!yolo detect train data=/content/data.yaml model=yolov8n.pt epochs=50 imgsz=640

100% 6.23M/6.23M [00:00<00:00, 204MB/s]
Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, forma

#### test model

In [5]:
from ultralytics import YOLO
# TODO test your model with test img in dataset
model = YOLO("/content/runs/detect/train/weights/best.pt")
model.predict("/content/valid/images/youtube-13_jpg.rf.97ba6aae2001f56d2642052799e4ea90.jpg", save=True, imgsz=640, conf=0.5)


image 1/1 /content/valid/images/youtube-13_jpg.rf.97ba6aae2001f56d2642052799e4ea90.jpg: 640x640 7 cars, 1 truck, 9.9ms
Speed: 2.1ms preprocess, 9.9ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bus', 1: 'car', 2: 'truck'}
 orig_img: array([[[143, 146, 137],
         [144, 145, 136],
         [156, 154, 146],
         ...,
         [133, 143, 143],
         [178, 188, 188],
         [175, 187, 187]],
 
        [[146, 149, 140],
         [147, 148, 139],
         [151, 149, 141],
         ...,
         [134, 144, 144],
         [185, 195, 195],
         [185, 197, 197]],
 
        [[148, 151, 142],
         [153, 154, 145],
         [147, 145, 137],
         ...,
         [133, 143, 143],
         [188, 198, 198],
         [190, 202, 202]],
 
        ...,
 
        [[ 88,  71,  74],
         [ 98,  81,  84],
         [108,  91,  94],
         ...,
         [101,  94,  97],
         [101,  94,  97],
         [101,  94,  97]],
 
        [[ 83,  66,  69],
         [ 96,  79,  82],
         [107,  90,  93],
         ...,
         [

## export yolov8 to trt

In [6]:
from pathlib import Path
# TODO your train dir ex. runs/detect/train/
train_path = Path("/content/runs/detect/train")
model_path = train_path / "weights"
pt_file = str(model_path / "best.pt")
onnx_file = str(model_path / "best.onnx")
trt_file = str(model_path / "best.engine")

In [7]:
# tensorRT engine
!yolo export model={pt_file} format=onnx

Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 245.8 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 8.0s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 8.7s, saved as '/content/runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (10.8s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val 

In [8]:
# download onnx
from google.colab import files
files.download(onnx_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>